In [6]:
import numpy as np
import ggseg
import pickle

from matplotlib import pyplot as plt 
import pandas as pd 
import plotly.express as px
import plotly.io as pio
import plotly.graph_objs as go
import plotly.offline as pyo

from rpy2 import robjects
from rpy2.robjects.packages import importr, data
import rpy2.robjects.lib.ggplot2 as ggplot2
from rpy2.robjects import pandas2ri

# imports the base module for R.
base = importr("base")

utils = importr("utils")
ggseg3d = importr("ggseg3d")
ggseg = importr("ggseg")
ggplot2 = importr("ggplot2")
dplyr = importr("dplyr")
tidyr = importr("tidyr")

In [29]:
# Import Pikle File
read_input_file = open('data/ADC_FTLD_subtypes_agecorrected_zscore_final.pickle','rb')
load_inputs = pickle.load(read_input_file)
read_input_file.close()

T, S, X = load_inputs

## New file

In [82]:
# Import Pikle File
read_input_file = open('data/EDADS_subtype_timelines_agecorrected_opt.pickle','rb')
load_inputs = pickle.load(read_input_file)
read_input_file.close()

T, S, X = load_inputs

/Users/macos/opt/anaconda3/envs/umc/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning:

Trying to unpickle estimator MinCovDet from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations



In [83]:
T.biomarker_labels

['Temporal_lobe',
 'Superior_frontal_gyrus',
 'Middle_frontal_gyrus',
 'Inferior_frontal_gyrus',
 'Gyrus_rectus',
 'Orbitofrontal_gyri',
 'Precentral_gyrus',
 'Postcentral_gyrus',
 'Superior_parietal_gyrus',
 'Inferolateral_remainder_of_parietal_lobe',
 'Lateral_remainder_of_occipital_lobe',
 'Lingual_gyrus',
 'Insula',
 'Gyrus_cinguli_anterior_part',
 'Gyrus_cinguli_posterior_part',
 'Parahippocampal_and_ambient_gyri',
 'Thalamus',
 'Caudate',
 'Putamen',
 'Hippocampus',
 'Amygdala',
 'Accumbens-area']

## Region Mapping

In [96]:
def mapping_dk(hemi = False):
    # if hemi = True, regions are mapped separately to left and right hemisphere
    
    # LEFT
    list_cortical_left = ['Temporal_lobe_left','Superior_frontal_gyrus_left',
                          'Middle_frontal_gyrus_left','Inferior_frontal_gyrus_left', 
                          'Gyrus_rectus_left','Orbitofrontal_gyri_left','Precentral_gyrus_left',
                          'Postcentral_gyrus_left','Superior_parietal_gyrus_left', 
                          'Inferolateral_remainder_of_parietal_lobe_left',
                          'Lateral_remainder_of_occipital_lobe_left','Lingual_gyrus_left', 
                          'Insula_left','Gyrus_cinguli_anterior_part_left','Gyrus_cinguli_posterior_part_left',
                          'Parahippocampal_and_ambient_gyri_left']

    org_cortical_mapping_left = [['bankssts_left','transverse_temporal_left',
                                      'superior_temporal_left','temporal_pole_left','entorhinal_left',
                                      'middle_temporal_left','inferior_temporal_left','fusiform_left'], 
                                     ['superior_frontal_left','frontal_pole_left'], 
                                     ['caudal_middle_frontal_left','rostral_middle_frontal_left'], 
                                     ['pars_opercularis_left','pars_orbitalis_left','pars_triangularis_left'], 
                                     ['medial_orbitofrontal_left'], ['lateral_orbitofrontal_left'], 
                                     ['precentral_left','paracentral_left'], ['postcentral_left'], 
                                     ['superior_parietal_left','precuneus_left','cuneus_left','pericalcarine_left'], 
                                 ['inferior_parietal_left','supramarginal_left'], 
                                     ['lateral_occipital_left'], 
                                     ['lingual_left'], ['insula_left'], ['caudal_anterior_cingulate_left','rostral_anterior_cingulate_left'], 
                                     ['posterior_cingulate_left','isthmus_cingulate_left'], 
                                     ['parahippocampal_left']]
    
    # RIGHT
    list_cortical_right = ['Temporal_lobe_right','Superior_frontal_gyrus_right','Middle_frontal_gyrus_right',
                           'Inferior_frontal_gyrus_right', 'Gyrus_rectus_right','Orbitofrontal_gyri_right',
                           'Precentral_gyrus_right','Postcentral_gyrus_right','Superior_parietal_gyrus_right', 
                           'Inferolateral_remainder_of_parietal_lobe_right','Lateral_remainder_of_occipital_lobe_right',
                           'Lingual_gyrus_right', 'Insula_right','Gyrus_cinguli_anterior_part_right',
                           'Gyrus_cinguli_posterior_part_right','Parahippocampal_and_ambient_gyri_right']
    
    org_cortical_mapping_right = [['bankssts_right','transverse_temporal_right',
                                  'superior_temporal_right','temporal_pole_right','entorhinal_right',
                                  'middle_temporal_right','inferior_temporal_right','fusiform_right'], 
                                 ['superior_frontal_right','frontal_pole_right'], 
                                 ['caudal_middle_frontal_right','rostral_middle_frontal_right'], 
                                 ['pars_opercularis_right','pars_orbitalis_right','pars_triangularis_right'], 
                                 ['medial_orbitofrontal_right'], ['lateral_orbitofrontal_right'], 
                                 ['precentral_right','paracentral_right'], ['postcentral_right'], 
                                 ['superior_parietal_right','precuneus_right','cuneus_right','pericalcarine_right'], 
                                  ['inferior_parietal_right','supramarginal_right'], 
                                 ['lateral_occipital_right'], 
                                 ['lingual_right'], ['insula_right'], ['caudal_anterior_cingulate_right','rostral_anterior_cingulate_right'], 
                                 ['posterior_cingulate_right','isthmus_cingulate_right'], 
                                 ['parahippocampal_right']]
    
    # BOTH HEMISPHERES
    list_cortical = ['Temporal_lobe','Superior_frontal_gyrus','Middle_frontal_gyrus','Inferior_frontal_gyrus', 
                     'Gyrus_rectus','Orbitofrontal_gyri','Precentral_gyrus','Postcentral_gyrus','Superior_parietal_gyrus', 
                     'Inferolateral_remainder_of_parietal_lobe','Lateral_remainder_of_occipital_lobe',
                     'Lingual_gyrus', 'Insula','Gyrus_cinguli_anterior_part','Gyrus_cinguli_posterior_part',
                     'Parahippocampal_and_ambient_gyri']

    org_cortical_mapping = [['bankssts_left','transverse_temporal_left',
                              'superior_temporal_left','temporal_pole_left','entorhinal_left',
                              'middle_temporal_left','inferior_temporal_left','fusiform_left','bankssts_right','transverse_temporal_right',
                          'superior_temporal_right','temporal_pole_right','entorhinal_right',
                          'middle_temporal_right','inferior_temporal_right','fusiform_right'],                             
                         ['superior_frontal_left','frontal_pole_left','superior_frontal_right','frontal_pole_right'], 
                         ['caudal_middle_frontal_left','rostral_middle_frontal_left','caudal_middle_frontal_right','rostral_middle_frontal_right'], 
                         ['pars_opercularis_left','pars_orbitalis_left','pars_triangularis_left','pars_opercularis_right','pars_orbitalis_right','pars_triangularis_right'], 
                         ['medial_orbitofrontal_left','medial_orbitofrontal_right'], ['lateral_orbitofrontal_left','lateral_orbitofrontal_right'], 
                         ['precentral_left','paracentral_left','precentral_right','paracentral_right'], ['postcentral_left','postcentral_right'], 
                         ['superior_parietal_left','precuneus_left','superior_parietal_right','precuneus_right','cuneus_left','pericalcarine_left','cuneus_right','pericalcarine_right'], ['inferior_parietal_left','supramarginal_left','inferior_parietal_right','supramarginal_right'], 
                         ['lateral_occipital_left','lateral_occipital_right'],  
                         ['lingual_left','lingual_right'], ['insula_left','insula_right'], ['caudal_anterior_cingulate_left','rostral_anterior_cingulate_left','caudal_anterior_cingulate_right','rostral_anterior_cingulate_right'], 
                         ['posterior_cingulate_left','isthmus_cingulate_left','posterior_cingulate_right','isthmus_cingulate_right'], 
                         ['parahippocampal_left','parahippocampal_right']]

    dic = {}
    
    if hemi == False:
        for idx, i in enumerate(list_cortical):
            dic[i.lower()] = org_cortical_mapping[idx] 
            
    elif hemi == True:
        list_cortical = list_cortical_left + list_cortical_right
        list_mapping = org_cortical_mapping_left + org_cortical_mapping_right
    
        for idx, i in enumerate(list_cortical):
            dic[i.lower()] = list_mapping[idx] 
        
    return dic
    

In [97]:
mapped_dict = mapping_dk()
# mapped_dict

## DK-atlas

In [98]:
def dk_df_3D(T,S, mapped_dict, subtype_labels = None, subtype = None):
    
    """
    Creates a dictionary, which can be used as input to ggseg3d() function
    :param T: dataframe from dk_dataframe() function
    :param S: chosen subtype
    :param mapped_dict: a dictionary with key: values --> T.biomarker_labels: list(DK-labels)
    :param subtype: name or index of the subtype from subtype_lables (optional, choses first available subtype as default)  
    :param subtype_labels: a list with names of the subtypes (optional)
    :return: dictionary with scores for each DK region for chosen subtype
    """
    
    unique_subtypes = np.unique(S['subtypes'][~np.isnan(S['subtypes'])])
    if subtype_labels is None:
        subtype_labels = {f'Subtype {i}': i for i in range(len(unique_subtypes))}
        if subtype is None:
            subtype = next(iter(subtype_labels))
    elif subtype is None:
        subtype = subtype_labels[0]  
        
    # clean names from capital letters
    labels = list(map(lambda x: x.lower(), T.biomarker_labels))
    labels_dict = {num: label.lower() for num, label in enumerate(labels)}
    
    order = T.sequence_model['ordering'][subtype_labels[subtype]]
    
    labels_ordered = []
    for o in order:
        labels_ordered.append(labels_dict[o])   
           
    dic = dict(zip(labels_ordered, T.sequence_model['event_centers'][subtype_labels[subtype]]))
                    
    # flat lost of dict values (single list of DK-labels)
    dk_flat = [x for v in mapped_dict.values() for x in v]
       
        
    hemi = []
    for idx, region in enumerate(dk_flat):
        if '_left' in region:
            hemi.append('left')
            dk_flat[idx]=dk_flat[idx].replace('_left','')
            dk_flat[idx]=dk_flat[idx].replace('_',' ')
        elif '_right' in region:
            hemi.append('right')
            dk_flat[idx]=dk_flat[idx].replace('_right','')
            dk_flat[idx]=dk_flat[idx].replace('_',' ')
        else:
            hemi.append('subcort')
            
    
    #Match T.biomarker_labels to DK labels
    list_plot = list()
    for key in mapped_dict.keys():
        for item in mapped_dict[key]:
            list_plot.append(dic[key])

    dic_dk = {'region': dk_flat, 'hemi':hemi, 'p': list_plot}
    df = pd.DataFrame(dic_dk)
    
    return df

In [105]:
dk = dk_df_3D(T, S, mapped_dict = mapped_dict, subtype = 'Subtype 3')
dk
# dk.to_csv('data/dk_R_Subtype 3.csv', index = False)

In [106]:
pd.read_csv("data/dk_R_Subtype 3.csv") 

,region,hemi,p
0,bankssts,left,0.290318
1,transverse temporal,left,0.290318
2,superior temporal,left,0.290318
3,temporal pole,left,0.290318
4,entorhinal,left,0.290318
...,...,...,...
63,isthmus cingulate,left,0.500888
64,posterior cingulate,right,0.500888
65,isthmus cingulate,right,0.500888
66,parahippocampal,left,0.657890


In [110]:
unique_subtypes = np.unique(S['subtypes'][~np.isnan(S['subtypes'])])
subtype_labels = {f'Subtype {i}': i for i in range(len(unique_subtypes))}
subtype_labels

{'Subtype 0': 0, 'Subtype 1': 1, 'Subtype 2': 2, 'Subtype 3': 3}

## Aseg atlas

In [115]:
def aseg_df_3D(T, S, subtype_labels = None, subtype = None):
    
    """
    Creates a dictionary, which can be used as input to ggseg.plot_dk() function
    :param T: dataframe from dk_dataframe() function
    :param S: chosen subtype
    :param subtype_labels: a list with names of the subtypes (optional)
    :param subtype: name or index of the subtype from subtype_lables (optional, choses first available subtype as default)  
    :return: dictionary with scores for each DK region for chosen subtype
    """

    unique_subtypes = np.unique(S['subtypes'][~np.isnan(S['subtypes'])])
    if subtype_labels is None:
        subtype_labels = {f'Subtype {i}': i for i in range(len(unique_subtypes))}
        if subtype is None:
            subtype = next(iter(subtype_labels))
    elif subtype is None:
        subtype = subtype_labels[0]
        
    labels = T.biomarker_labels
    labels_dict = {num: label for num, label in enumerate(labels)}
    order = T.sequence_model['ordering'][subtype_labels[subtype]]
        
    labels_ordered = []
    for o in order:
        labels_ordered.append(labels_dict[o])
        
    regions= labels_ordered*2
    for l in range(len(labels_ordered)):
            regions[l]='Left-' + regions[l]
    for l in range(len(labels_ordered), 2*len(labels_ordered)):
        regions[l]='Right-' + regions[l]

    values = list(T.sequence_model['event_centers'][subtype_labels[subtype]]) + list(T.sequence_model['event_centers'][subtype_labels[subtype]])                                                                   
                        
           
    dic_aseg = {'region': regions, 'p': values}
    df = pd.DataFrame(dic_aseg)
        
    return df

In [122]:
aseg = aseg_df_3D(T,S, subtype = 'Subtype 3')
aseg

# aseg.to_csv('data/aseg_R_Subtype 3.csv', index = False)

In [123]:
pd.read_csv("data/aseg_R_Subtype 3.csv") 

,region,p
0,Left-Superior_parietal_gyrus,0.056255
1,Left-Inferolateral_remainder_of_parietal_lobe,0.090823
2,Left-Postcentral_gyrus,0.184063
3,Left-Lateral_remainder_of_occipital_lobe,0.213270
4,Left-Precentral_gyrus,0.285589
5,Left-Temporal_lobe,0.290318
6,Left-Thalamus,0.369099
7,Left-Middle_frontal_gyrus,0.379467
8,Left-Putamen,0.458347
9,Left-Lingual_gyrus,0.458347
